In [1]:
import pandas
import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word

In [5]:
uniqWords = []
tupleIndex = 0
sentences = []
for tuple in df.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
testSentences = []
for tuple in test.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    testSentences.append(wordsList)

In [6]:
import multiprocessing
from gensim.models import Word2Vec
uniqWords = set(uniqWords)
model = Word2Vec(sentences=sentences,min_count=1,size=32)

In [7]:
print(model.wv.most_similar('конец'))
print(model.wv.most_similar('фильм'))
print(model.wv.most_similar('он'))
print(model.wv.most_similar('режиссер'))
print(model.wv.most_similar('герой'))

[('амели', 0.9998214244842529), ('начинать', 0.999721109867096), ('всегда', 0.9996988773345947), ('реальность', 0.9996824264526367), ('времени', 0.9996778964996338), ('тут', 0.9996770620346069), ('смысл', 0.9996719360351562), ('тоже', 0.9996703863143921), ('теперь', 0.9996680021286011), ('время', 0.9996675252914429)]
[('посмотреть', 0.9988816380500793), ('ни', 0.9987820982933044), ('чем', 0.998660683631897), ('только', 0.9985199570655823), ('шедевр', 0.9984079599380493), ('кто', 0.9983744621276855), ('забыть', 0.9982613921165466), ('ещё', 0.9982547163963318), ('единственное', 0.9982055425643921), ('несмотря', 0.9981734752655029)]
[('много', 0.9996849298477173), ('все', 0.9996049404144287), ('ты', 0.9995788931846619), ('понимать', 0.9995312690734863), ('мало', 0.9995127320289612), ('нет', 0.9995114803314209), ('мы', 0.9995007514953613), ('она', 0.9994980692863464), ('вот', 0.999492883682251), ('даже', 0.9994525909423828)]
[('знаменитый', 0.9984984993934631), ('отличный', 0.9984617829322

In [8]:
def getMiddleValue(array):
    sum = 0
    for i in array:
        sum += i
    return sum/len(array)
print(model.wv['блут'])

[ 0.02639376 -0.06133854 -0.03379047  0.026828    0.00297044  0.04419322
  0.02815407  0.04377708 -0.01719869  0.08568732  0.0030121  -0.03489447
  0.03668172  0.00902367 -0.04218019  0.02299653 -0.01711602 -0.05457292
  0.01265272  0.02959881 -0.02873543  0.02561061 -0.00200684  0.04097864
 -0.04696902  0.03350537 -0.00923586 -0.04765229  0.03512321 -0.0662681
  0.01541122 -0.07415998]


In [9]:
X_train = pandas.DataFrame(columns = uniqWords)
index = 0
for comment in sentences:
    index += 1
    commentLength = len(comment)
    arrayDict = dict.fromkeys(uniqWords, 0)
    for word in comment:
        vector = model.wv[word]
        mid_vector = list(map(lambda x: x/commentLength, vector))
        value = getMiddleValue(mid_vector)
        arrayDict[word] = value
    X_train.loc[index] = list(arrayDict.values())


In [10]:
Y_train = df["label"]
Y_test = test["label"]

In [11]:
from sklearn.ensemble import RandomForestClassifier
randomForestCLF = RandomForestClassifier(max_depth=20, random_state=0).fit(X_train, Y_train)

In [12]:
X_test = pandas.DataFrame(columns = uniqWords)
index = 0
for comment in testSentences:
    index += 1
    commentLength = len(comment)
    arrayDict = dict.fromkeys(uniqWords, 0)
    for word in comment:
        if word in list(model.wv.vocab.keys()):
            vector = model.wv[word]
            mid_vector = list(map(lambda x: x/commentLength, vector))
            value = getMiddleValue(mid_vector)
            arrayDict[word] = value
        else:
            arrayDict[word] = 0
    X_test.loc[index] = list(arrayDict.values())

In [13]:
from sklearn.metrics import precision_recall_fscore_support
randomForestPredict = randomForestCLF.predict(X_test)
randomForestMetric = precision_recall_fscore_support(test["label"].values, randomForestPredict, average='weighted')

In [14]:
print(randomForestMetric)

(0.39940518890483023, 0.4111111111111111, 0.3939054137223248, None)
